# HW1: Frame-Level Speech Recognition

Running Instruction: 
Just run this notebook.

Architectures tried: 6-8 layers, including diamond, cylinder, pyrimaid, stack 2 pyrimaids

Number of epochs trained for: 10

Hyperparameters used: 

- Learning rate scheduler: CosineAnnealing, ReduceOnPlateu

- Weight initialization: Kaiming and Xavier

- Context: 20, 30

- Dropout rate: 0.1, 0.2

- Batch Size: 1024, 2048, 8192

- Batch Normalization

- Weight Decay: 0.01, 0.001, None

- Activation function: Relu, LeakyRelu, Gelu


Best combination: 
Architecture: 8 layers of diamond with Gelu activation and batchnormalization. 
Drop out rate : 0.2
Learning rate scheduler: CosineAnnealing
Weight initialization: Kaiming
Context: 20
Batch Size: 8192

Network(
  (model): Sequential(
    (0): Linear(in_features=1107, out_features=512, bias=True)
    (1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): GELU(approximate='none')
    (3): Linear(in_features=512, out_features=1024, bias=True)
    (4): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): GELU(approximate='none')
    (6): Linear(in_features=1024, out_features=2048, bias=True)
    (7): BatchNorm1d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): GELU(approximate='none')
    (9): Dropout(p=0.2, inplace=False)
    (10): Linear(in_features=2048, out_features=4096, bias=True)
    (11): BatchNorm1d(4096, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): GELU(approximate='none')
    (13): Dropout(p=0.2, inplace=False)
    (14): Linear(in_features=4096, out_features=2048, bias=True)
    (15): BatchNorm1d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (16): GELU(approximate='none')
    (17): Linear(in_features=2048, out_features=1024, bias=True)
    (18): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (19): GELU(approximate='none')
    (20): Dropout(p=0.2, inplace=False)
    (21): Linear(in_features=1024, out_features=512, bias=True)
    (22): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (23): GELU(approximate='none')
    (24): Dropout(p=0.2, inplace=False)
    (25): Linear(in_features=512, out_features=40, bias=True)
  )

data loading scheme: 
Followed the bootcamp, I counted the mfcc length and initialized the zero matrix with the length and padding process before load data to fill the zeroes.

# Libraries

In [ ]:
!pip install torchsummaryX wandb --quiet

In [ ]:
import torch
import numpy as np
from torchsummaryX import summary
import sklearn
import gc
import zipfile
import pandas as pd
from tqdm.auto import tqdm
import os
import datetime
import wandb
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Device: ", device)

Device:  cuda


In [ ]:
### If you are using colab, you can import google drive to save model checkpoints in a folder
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
### PHONEME LIST
PHONEMES = [
            '[SIL]',   'AA',    'AE',    'AH',    'AO',    'AW',    'AY',  
            'B',     'CH',    'D',     'DH',    'EH',    'ER',    'EY',
            'F',     'G',     'HH',    'IH',    'IY',    'JH',    'K',
            'L',     'M',     'N',     'NG',    'OW',    'OY',    'P',
            'R',     'S',     'SH',    'T',     'TH',    'UH',    'UW',
            'V',     'W',     'Y',     'Z',     'ZH',    '[SOS]', '[EOS]']

In [ ]:
Inverse_PHONEMES_MAP = {}
for i,p in enumerate(PHONEMES):
  Inverse_PHONEMES_MAP[i] = p

# Kaggle

This section contains code that helps you install kaggle's API, creating kaggle.json with you username and API key details. Make sure to input those in the given code to ensure you can download data from the competition successfully.

In [ ]:
!pip install --upgrade --force-reinstall --no-deps kaggle==1.5.8
!mkdir /root/.kaggle

with open("/root/.kaggle/kaggle.json", "w+") as f:
    f.write('{"username":"yunxingu","key":"b0d791b8d8e9237e047196d9e5f3c070"}') 
    # Put your kaggle username & key here

!chmod 600 /root/.kaggle/kaggle.json

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 KB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for kaggle: filename=kaggle-1.5.8-py3-none-any.whl size=73274 sha256=3093cb2393abbf5c6497ea086a2f03c8d9885a6cf535a8576a20fe9513b6e0ad
  Stored in directory: /root/.cache/pip/wheels/f3/67/7b/a6d668747974998471d29b230e7221dd01330ac34faebe4af4
Successfully built kaggle
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.5.12
    Uninstalling kaggle-1.5.12:
      Successfully uninstalled kaggle-1.5.12


In [ ]:
# commands to download data from kaggle

!kaggle competitions download -c 11-785-s23-hw1p2
!mkdir '/content/data'

!unzip -qo '11-785-s23-hw1p2.zip' -d '/content/data'

100% 16.0G/16.0G [01:18<00:00, 206MB/s]
100% 16.0G/16.0G [01:18<00:00, 219MB/s]


In [ ]:
# !mkdir '/content/data'

# !unzip -qo '/content/drive/MyDrive/11685/hw1p2/drive-download-20230203T170752Z-001.zip' -d '/content/data'

In [ ]:
import random

# Dataset

This section covers the dataset/dataloader class for speech data. You will have to spend time writing code to create this class successfully. We have given you a lot of comments guiding you on what code to write at each stage, from top to bottom of the class. Please try and take your time figuring this out, as it will immensely help in creating dataset/dataloader classes for future homeworks.

Before running the following cells, please take some time to analyse the structure of data. Try loading a single MFCC and its transcipt, print out the shapes and print out the values. Do the transcripts look like phonemes?

In [ ]:
# Dataset class to load train and validation data

class AudioDataset(torch.utils.data.Dataset):

    def __init__(self, root, phonemes = PHONEMES, context=0, partition= "train-clean-100"): # Feel free to add more arguments

        self.context    = context
        self.phonemes   = phonemes[:-2]
        PHONEMES_MAP = {}
        for i,p in enumerate(self.phonemes):
          PHONEMES_MAP[p] = i
        
        
        # TODO: MFCC directory - use partition to acces train/dev directories from kaggle data using root
        self.mfcc_dir       = root+partition+'/mfcc/' 
        # TODO: Transcripts directory - use partition to acces train/dev directories from kaggle data using root
        self.transcript_dir = root+partition+'/transcript/' 

        # TODO: List files in sefl.mfcc_dir using os.listdir in sorted order
        mfcc_names          =  sorted(os.listdir(self.mfcc_dir))
        # TODO: List files in self.transcript_dir using os.listdir in sorted order
        transcript_names    =  sorted(os.listdir(self.transcript_dir))

        # Making sure that we have the same no. of mfcc and transcripts
        assert len(mfcc_names) == len(transcript_names) 

        length = len(mfcc_names)

        T = 0

        for i in range(length):
          mfcc = np.load(self.mfcc_dir + mfcc_names[i])
          T += mfcc.shape[0]
        self.mfccs = np.zeros((T+2*context, mfcc.shape[1]), dtype = np.float32)
        #  completed padding here
        self.transcripts = np.zeros((T,), dtype = np.uint8)

        cx, cy = context, 0

        # TODO: Iterate through mfccs and transcripts
        for i in range(length):
        #   Load a single mfcc
            mfcc        = np.load(self.mfcc_dir + mfcc_names[i])
        #   Do Cepstral Normalization of mfcc (explained in writeup)
            normalized_mfcc = (mfcc - np.mean(mfcc, axis = 0))/np.std(mfcc, axis = 0)
        #   Load the corresponding transcript
            transcript  = np.load(self.transcript_dir + transcript_names[i]) 
            transcript = transcript[1:-1] # Remove [SOS] and [EOS] from the transcript 
            
            # Note that SOS will always be in the starting and EOS at end, as the name suggests.
        #   Append each mfcc to self.mfcc, transcript to self.transcript
            transcript = [PHONEMES_MAP[p] for p in transcript]

            T_i = mfcc.shape[0]
            self.mfccs[cx: cx+T_i] = mfcc
            self.transcripts[cy: cy+T_i] = transcript
            cx += T_i
            cy += T_i      

        # NOTE:
        # Each mfcc is of shape T1 x 27, T2 x 27, ...
        # Each transcript is of shape (T1+2),)( (T2+2),) before removing [SOS] and [EOS]

        # Length of the dataset is now the length of concatenated mfccs/transcripts
        self.length = T

        # self.mfcc is an array of the format (Frames x Features). 
       
        
        # Now, if an element in self.transcript is 0, it means that it is 'SIL' (as per the above example)

    def __len__(self):
        return self.length

    def __getitem__(self, ind):

        ### Calculate starting timestep using offset and context (1 line)
        # start_i = index + self.offset - self.context
        start_i = ind - self.context
        
        ## Calculate ending timestep using offset and context (1 line)
        # end_i = index + self.offset + self.context + 1
        end_i = ind + self.context + 1

        
        # TODO: Based on context and offset, return a frame at given index with context frames to the left, and right.
        # frames = self.mfccs[start_i:end_i]
        frames = self.mfccs[ind:ind+2*self.context+1]
        # After slicing, you get an array of shape 2*context+1 x 27. But our MLP needs 1d data and not 2d.
        frames = frames.flatten() # TODO: Flatten to get 1d data

        frames      = torch.FloatTensor(frames) # Convert to tensors
        phonemes    = torch.tensor(self.transcripts[ind])       

        return frames, phonemes

In [ ]:
# # Original Dataset class to load train and validation data

# class AudioDataset(torch.utils.data.Dataset):

#     def __init__(self, root, phonemes = PHONEMES, context=0, partition= "train-clean-100"): # Feel free to add more arguments

#         self.context    = context
#         self.phonemes   = phonemes[:-2]
        
#         # TODO: MFCC directory - use partition to acces train/dev directories from kaggle data using root
#         self.mfcc_dir       = root+partition+'/mfcc/' 
#         # TODO: Transcripts directory - use partition to acces train/dev directories from kaggle data using root
#         self.transcript_dir = root+partition+'/transcript/' 

#         # TODO: List files in sefl.mfcc_dir using os.listdir in sorted order
#         mfcc_names          =  sorted(os.listdir(self.mfcc_dir))
#         # TODO: List files in self.transcript_dir using os.listdir in sorted order
#         transcript_names    =  sorted(os.listdir(self.transcript_dir))

#         # Making sure that we have the same no. of mfcc and transcripts
#         assert len(mfcc_names) == len(transcript_names) 

#         self.mfccs, self.transcripts = [], []

#         # TODO: Iterate through mfccs and transcripts
#         for i in range(len(mfcc_names)):
#         #   Load a single mfcc
#             mfcc        = np.load(self.mfcc_dir + mfcc_names[i])
#         #   Do Cepstral Normalization of mfcc (explained in writeup)

#             normalized_mfcc = (mfcc - np.mean(mfcc, axis = 0))/np.std(mfcc, axis = 0)
#         #   Load the corresponding transcript
#             transcript  = np.load(self.transcript_dir + transcript_names[i]) 
#             transcript = transcript[1:-1]

#             # Remove [SOS] and [EOS] from the transcript 
#             # (Is there an efficient way to do this without traversing through the transcript?)
#             # Note that SOS will always be in the starting and EOS at end, as the name suggests.
#         #   Append each mfcc to self.mfcc, transcript to self.transcript
#             self.mfccs.append(mfcc)
#             self.transcripts.append(transcript)        

#         # NOTE:
#         # Each mfcc is of shape T1 x 27, T2 x 27, ...
#         # Each transcript is of shape (T1+2),)( (T2+2),) before removing [SOS] and [EOS]

#         # TODO: Concatenate all mfccs in self.mfccs such that 
#         # the final shape is T x 27 (Where T = T1 + T2 + ...) 
      
#         self.mfccs  = np.concatenate(self.mfccs,  axis = 0)

#         # TODO: Concatenate all transcripts in self.transcripts such that 
#         # the final shape is (T,) meaning, each time step has one phoneme output
#         self.transcripts    = np.concatenate(self.transcripts,  axis = 0)
#         # Hint: Use numpy to concatenate

#         # Length of the dataset is now the length of concatenated mfccs/transcripts
#         self.length = len(self.mfccs)

#         # Take some time to think about what we have done. 
#         # self.mfcc is an array of the format (Frames x Features). 
#         # Our goal is to recognize phonemes of each frame
#         # From hw0, you will be knowing what context is. 
#         # We can introduce context by padding zeros on top and bottom of self.mfcc
#         self.mfccs = np.pad(self.mfccs, ((context,context),(0,0)), 'constant', constant_values = (0,0))

#         # The available phonemes in the transcript are of string data type
#         # But the neural network cannot predict strings as such. 
#         # Hence, we map these phonemes to integers

#         # TODO: Map the phonemes to their corresponding list indexes in self.phonemes
#         PHONEMES_MAP = {}
#         for i,p in enumerate(self.phonemes):
#           PHONEMES_MAP[p] = i
#         self.transcripts = [PHONEMES_MAP[p] for p in self.transcripts]
#         # Now, if an element in self.transcript is 0, it means that it is 'SIL' (as per the above example)

#     def __len__(self):
#         return self.length

#     def __getitem__(self, ind):

#         ### Calculate starting timestep using offset and context (1 line)
#         # start_i = index + self.offset - self.context
#         start_i = ind - self.context
        
#         ## Calculate ending timestep using offset and context (1 line)
#         # end_i = index + self.offset + self.context + 1
#         end_i = ind + self.context + 1

        
#         # TODO: Based on context and offset, return a frame at given index with context frames to the left, and right.
#         # frames = self.mfccs[start_i:end_i]
#         frames = self.mfccs[ind:ind+2*self.context+1]
#         # After slicing, you get an array of shape 2*context+1 x 27. But our MLP needs 1d data and not 2d.
#         frames = frames.flatten() # TODO: Flatten to get 1d data

#         frames      = torch.FloatTensor(frames) # Convert to tensors
#         phonemes    = torch.tensor(self.transcripts[ind])       

#         return frames, phonemes

In [ ]:
from zmq import Context
class AudioTestDataset(torch.utils.data.Dataset):
    def __init__(self, root, phonemes = PHONEMES, context=0, partition= "test-clean"): # Feel free to add more arguments

        self.context    = context
        self.phonemes   = phonemes[:-2]
        
        # TODO: MFCC directory - use partition to acces train/dev directories from kaggle data using root
        self.mfcc_dir       = root+partition+'/mfcc/' 

        # TODO: List files in sefl.mfcc_dir using os.listdir in sorted order
        mfcc_names          =  sorted(os.listdir(self.mfcc_dir))

        self.mfccs = []

        # TODO: Iterate through mfccs and transcripts
        for i in range(len(mfcc_names)):
        #   Load a single mfcc
            mfcc        = np.load(self.mfcc_dir + mfcc_names[i])
        #   Do Cepstral Normalization of mfcc (explained in writeup)

            normalized_mfcc = (mfcc - np.mean(mfcc))/np.std(mfcc)

            # Remove [SOS] and [EOS] from the transcript 
            # (Is there an efficient way to do this without traversing through the transcript?)
            # Note that SOS will always be in the starting and EOS at end, as the name suggests.
        #   Append each mfcc to self.mfcc, transcript to self.transcript
            self.mfccs.append(mfcc)

        # NOTE:
        # Each mfcc is of shape T1 x 27, T2 x 27, ...
        # Each transcript is of shape (T1+2),)( (T2+2),) before removing [SOS] and [EOS]

        # TODO: Concatenate all mfccs in self.mfccs such that 
        # the final shape is T x 27 (Where T = T1 + T2 + ...) 
      
        self.mfccs  = np.concatenate(self.mfccs,  axis = 0)

        # TODO: Concatenate all transcripts in self.transcripts such that 
        # the final shape is (T,) meaning, each time step has one phoneme output
        # Hint: Use numpy to concatenate

        # Length of the dataset is now the length of concatenated mfccs/transcripts
        self.length = len(self.mfccs)

        # Take some time to think about what we have done. 
        # self.mfcc is an array of the format (Frames x Features). 
        # Our goal is to recognize phonemes of each frame
        # From hw0, you will be knowing what context is. 
        # We can introduce context by padding zeros on top and bottom of self.mfcc
        self.mfccs = np.pad(self.mfccs, ((self.context,self.context), (0,0)), 'constant', constant_values = (0,0))

        # The available phonemes in the transcript are of string data type
        # But the neural network cannot predict strings as such. 
        # Hence, we map these phonemes to integers


        # Now, if an element in self.transcript is 0, it means that it is 'SIL' (as per the above example)

    def __len__(self):
        return self.length

    def __getitem__(self, ind):

        ### Calculate starting timestep using offset and context (1 line)
        # start_i = index + self.offset - self.context
        start_i = ind - self.context
        
        ## Calculate ending timestep using offset and context (1 line)
        # end_i = index + self.offset + self.context + 1
        end_i = ind + self.context + 1

        
        # TODO: Based on context and offset, return a frame at given index with context frames to the left, and right.
        # frames = self.mfccs[start_i:end_i]
        frames = self.mfccs[ind:ind+2*self.context+1]
        # After slicing, you get an array of shape 2*context+1 x 27. But our MLP needs 1d data and not 2d.
        frames = frames.flatten() # TODO: Flatten to get 1d data

        frames      = torch.FloatTensor(frames) # Convert to tensors
        

        return frames

    # TODO: Create a test dataset class similar to the previous class but you dont have transcripts for this
    # Imp: Read the mfccs in sorted order, do NOT shuffle the data here or in your dataloader.

# Parameters Configuration

Storing your parameters and hyperparameters in a single configuration dictionary makes it easier to keep track of them during each experiment. It can also be used with weights and biases to log your parameters for each experiment and keep track of them across multiple experiments. 

In [ ]:
config = {
    'epochs'        : 10,
    'batch_size'    : 8192,
    'context'       : 20,
    'init_lr'       : 1e-3,
    'dropout_rate': 0.2,
    'architecture'  : '2pyramid-8'
    # Add more as you need them - e.g dropout values, weight decay, scheduler parameters
}

# Create Datasets

In [ ]:
#TODO: Create a dataset object using the AudioDataset class for the training data 
train_data = AudioDataset('/content/data/11-785-s23-hw1p2/', phonemes = PHONEMES, context=config['context'], partition= "train-clean-360") 

# TODO: Create a dataset object using the AudioDataset class for the validation data 
val_data = AudioDataset('/content/data/11-785-s23-hw1p2/', phonemes = PHONEMES, context=config['context'], partition= "dev-clean") 

# TODO: Create a dataset object using the AudioTestDataset class for the test data 
test_data = AudioTestDataset('/content/data/11-785-s23-hw1p2/', phonemes = PHONEMES, context=config['context'], partition= "test-clean") 

In [ ]:
# val_data[1]

In [ ]:
# phs = [val_data[i][1].detach().numpy().item() for i in range(len(val_data))]

In [ ]:
# phs = np.array(phs)

In [ ]:
# np.sum(phs == 0)

In [ ]:
# len(phs)

In [ ]:
# np.unique(phs,return_counts=True)

In [ ]:
# Define dataloaders for train, val and test datasets
# Dataloaders will yield a batch of frames and phonemes of given batch_size at every iteration
# We shuffle train dataloader but not val & test dataloader. Why?

train_loader = torch.utils.data.DataLoader(
    dataset     = train_data, 
    num_workers = 4,
    batch_size  = config['batch_size'], 
    pin_memory  = True,
    shuffle     = True
)

val_loader = torch.utils.data.DataLoader(
    dataset     = val_data, 
    num_workers = 2,
    batch_size  = config['batch_size'],
    pin_memory  = True,
    shuffle     = False
)

test_loader = torch.utils.data.DataLoader(
    dataset     = test_data, 
    num_workers = 0, 
    batch_size  = config['batch_size'], 
    pin_memory  = True, 
    shuffle     = False
)


print("Batch size     : ", config['batch_size'])
print("Context        : ", config['context'])
print("Input size     : ", (2*config['context']+1)*27)
print("Output symbols : ", len(PHONEMES))

print("Train dataset samples = {}, batches = {}".format(train_data.__len__(), len(train_loader)))
print("Validation dataset samples = {}, batches = {}".format(val_data.__len__(), len(val_loader)))
print("Test dataset samples = {}, batches = {}".format(test_data.__len__(), len(test_loader)))

Batch size     :  8192
Context        :  20
Input size     :  1107
Output symbols :  42
Train dataset samples = 130453995, batches = 15925
Validation dataset samples = 1928204, batches = 236
Test dataset samples = 1934138, batches = 237


In [ ]:
# test_loader = torch.utils.data.DataLoader(
#     dataset     = test_data, 
#     num_workers = 0, 
#     batch_size  = config['batch_size'], 
#     pin_memory  = True, 
#     shuffle     = False
# )

In [ ]:
# Testing code to check if your data loaders are working
for i, data in enumerate(train_loader):
    frames, phoneme = data
    print(frames.shape, phoneme.shape)
    break

torch.Size([8192, 1107]) torch.Size([8192])


# Network Architecture


This section defines your network architecture for the homework. We have given you a sample architecture that can easily clear the very low cutoff for the early submission deadline.

In [ ]:
def weights_init(m):
    if isinstance(m, torch.nn.Linear):
        # torch.nn.init.xavier_uniform(m.weight)
        torch.nn.init.kaiming_normal_(m.weight)
        m.bias.data.fill_(0.01)


In [ ]:
# This architecture will make you cross the very low cutoff
# However, you need to run a lot of experiments to cross the medium or high cutoff
class Network(torch.nn.Module):

    def __init__(self, input_size, output_size):

        super(Network, self).__init__()

        self.model = torch.nn.Sequential(
            torch.nn.Linear(input_size, 512),
            torch.nn.BatchNorm1d(512),
            torch.nn.GELU(),
            torch.nn.Linear(512, 1024),
            torch.nn.BatchNorm1d(1024),
            torch.nn.GELU(),
            torch.nn.Linear(1024, 2048),
            torch.nn.BatchNorm1d(2048),
            torch.nn.GELU(),
            torch.nn.Dropout(config['dropout_rate']),
            torch.nn.Linear(2048, 3000),
            torch.nn.BatchNorm1d(3000),
            torch.nn.GELU(),
            torch.nn.Dropout(config['dropout_rate']),
            torch.nn.Linear(3000, 2048),
            torch.nn.BatchNorm1d(2048),
            torch.nn.GELU(),
            torch.nn.Linear(2048, 1024),
            torch.nn.BatchNorm1d(1024),
            torch.nn.GELU(),
            torch.nn.Dropout(config['dropout_rate']),
            torch.nn.Linear(1024, 512),
            torch.nn.BatchNorm1d(512),
            torch.nn.GELU(),
            torch.nn.Dropout(config['dropout_rate']),
            torch.nn.Linear(512, output_size)
        )
        self.model.apply(weights_init)      

    def forward(self, x):
        out = self.model(x)

        return out

# Define Model, Loss Function and Optimizer

Here we define the model, loss function, optimizer and optionally a learning rate scheduler. 

In [ ]:
INPUT_SIZE  = (2*config['context'] + 1) * 27 # Why is this the case?
model       = Network(INPUT_SIZE, len(train_data.phonemes)).to(device)
summary(model, frames.to(device))
# Check number of parameters of your network
# Remember, you are limited to 20 million parameters for HW1 (including ensembles)

                         Kernel Shape  Output Shape     Params  Mult-Adds
Layer                                                                    
0_model.Linear_0          [1107, 512]   [8192, 512]   567.296k   566.784k
1_model.BatchNorm1d_1           [512]   [8192, 512]     1.024k      512.0
2_model.GELU_2                      -   [8192, 512]          -          -
3_model.Linear_3          [512, 1024]  [8192, 1024]   525.312k   524.288k
4_model.BatchNorm1d_4          [1024]  [8192, 1024]     2.048k     1.024k
5_model.GELU_5                      -  [8192, 1024]          -          -
6_model.Linear_6         [1024, 2048]  [8192, 2048]    2.0992M  2.097152M
7_model.BatchNorm1d_7          [2048]  [8192, 2048]     4.096k     2.048k
8_model.GELU_8                      -  [8192, 2048]          -          -
9_model.Dropout_9                   -  [8192, 2048]          -          -
10_model.Linear_10       [2048, 4096]  [8192, 4096]  8.392704M  8.388608M
11_model.BatchNorm1d_11        [4096] 

/usr/local/lib/python3.8/dist-packages/torchsummaryX/torchsummaryX.py:101: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_sum = df.sum()


,Kernel Shape,Output Shape,Params,Mult-Adds
Layer,,,,
0_model.Linear_0,"[1107, 512]","[8192, 512]",567296.0,566784.0
1_model.BatchNorm1d_1,[512],"[8192, 512]",1024.0,512.0
2_model.GELU_2,-,"[8192, 512]",NaN,NaN
3_model.Linear_3,"[512, 1024]","[8192, 1024]",525312.0,524288.0
4_model.BatchNorm1d_4,[1024],"[8192, 1024]",2048.0,1024.0
5_model.GELU_5,-,"[8192, 1024]",NaN,NaN
6_model.Linear_6,"[1024, 2048]","[8192, 2048]",2099200.0,2097152.0
7_model.BatchNorm1d_7,[2048],"[8192, 2048]",4096.0,2048.0
8_model.GELU_8,-,"[8192, 2048]",NaN,NaN


In [ ]:
# pretrained_model = torch.load('/content/model.pt')

In [ ]:
criterion = torch.nn.CrossEntropyLoss() # Defining Loss function. 
# We use CE because the task is multi-class classification 

optimizer = torch.optim.Adam(model.parameters(), lr= config['init_lr']) #Defining Optimizer
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max = 20, eta_min = 1e-4)
# scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min')

# Recommended : Define Scheduler for Learning Rate, 
# including but not limited to StepLR, MultiStepLR, CosineAnnealingLR, ReduceLROnPlateau, etc. 
# You can refer to Pytorch documentation for more information on how to use them.

scaler = torch.cuda.amp.GradScaler()
# Is your training time very high? 
# Look into mixed precision training if your GPU (Tesla T4, V100, etc) can make use of it 
# Refer - https://pytorch.org/docs/stable/notes/amp_examples.html

# Training and Validation Functions

This section covers the training, and validation functions for each epoch of running your experiment with a given model architecture. The code has been provided to you, but we recommend going through the comments to understand the workflow to enable you to write these loops for future HWs.

In [ ]:
torch.cuda.empty_cache()
gc.collect()

1197

In [ ]:
# from torch.optim import lr_scheduler
def train(model, dataloader, optimizer, criterion):

    model.train()
    tloss, tacc = 0, 0 # Monitoring loss and accuracy
    batch_bar   = tqdm(total=len(train_loader), dynamic_ncols=True, leave=False, position=0, desc='Train')
    lr_schedule = []
    
    for i, (frames, phonemes) in enumerate(dataloader):
        
        ### Initialize Gradients
        optimizer.zero_grad()

        with torch.autocast(device_type='cuda', dtype=torch.float16):

          ### Move Data to Device (Ideally GPU)
          frames      = frames.to(device)
          phonemes    = phonemes.to(device)

          ### Forward Propagation
          logits  = model(frames)

          ### Loss Calculation
          loss    = criterion(logits, phonemes)

        ### Backward Propagation
        # loss.backward() 
        scaler.scale(loss).backward()
        
        ### Gradient Descent
        # optimizer.step() 
        scaler.step(optimizer)  

        tloss   += loss.item()
        tacc    += torch.sum(torch.argmax(logits, dim= 1) == phonemes).item()/logits.shape[0]

        batch_bar.set_postfix(loss="{:.04f}".format(float(tloss / (i + 1))), 
                              acc="{:.04f}%".format(float(tacc*100 / (i + 1))))
        batch_bar.update()
        # scaler.update()


        ### Release memory
        del frames, phonemes, logits
        torch.cuda.empty_cache()
  
    batch_bar.close()
    tloss   /= len(train_loader)
    tacc    /= len(train_loader)

    return tloss, tacc

In [ ]:
def eval(model, dataloader):

    model.eval() # set model in evaluation mode
    vloss, vacc = 0, 0 # Monitoring loss and accuracy
    batch_bar   = tqdm(total=len(val_loader), dynamic_ncols=True, position=0, leave=False, desc='Val')

    for i, (frames, phonemes) in enumerate(dataloader):

        ### Move data to device (ideally GPU)
        frames      = frames.to(device)
        phonemes    = phonemes.to(device)

        # makes sure that there are no gradients computed as we are not training the model now
        with torch.inference_mode(): 
            ### Forward Propagation
            logits  = model(frames)
            ### Loss Calculation
            loss    = criterion(logits, phonemes)

        vloss   += loss.item()
        vacc    += torch.sum(torch.argmax(logits, dim= 1) == phonemes).item()/logits.shape[0]
        
        # Do you think we need loss.backward() and optimizer.step() here?

        batch_bar.set_postfix(loss="{:.04f}".format(float(vloss / (i + 1))), 
                              acc="{:.04f}%".format(float(vacc*100 / (i + 1))))
        batch_bar.update()
    
        ### Release memory
        del frames, phonemes, logits
        torch.cuda.empty_cache()

    batch_bar.close()
    vloss   /= len(val_loader)
    vacc    /= len(val_loader)

    return vloss, vacc

# Weights and Biases Setup

This section is to enable logging metrics and files with Weights and Biases. Please refer to wandb documentationa and recitation 0 that covers the use of weights and biases for logging, hyperparameter tuning and monitoring your runs for your homeworks. Using this tool makes it very easy to show results when submitting your code and models for homeworks, and also extremely useful for study groups to organize and run ablations under a single team in wandb. 

We have written code for you to make use of it out of the box, so that you start using wandb for all your HWs from the beginning.

In [ ]:
wandb.login(key="eac70e317e1b0b232f1e96dda29ff11405e8b4bc") #API Key is in your wandb account, under settings (wandb.ai/settings)

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: silviagu (11785-team). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
# Create your wandb run
run = wandb.init(
    name    = "silvia-8diamonds-GELU-batchnorm-dropout-kaiming-Annealing-360", ### Wandb creates random run names if you skip this field, we recommend you give useful names
    reinit  = True, ### Allows reinitalizing runs when you re-run this cell
    #id     = "y28t31uz", ### Insert specific run id here if you want to resume a previous run
    #resume = "must", ### You need this to resume previous runs, but comment out reinit = True when using this
    project = "hw1p2", ### Project should be created in your wandb account 
    config  = config ### Wandb Config for your run
)

In [ ]:
### Save your model architecture as a string with str(model) 
model_arch  = str(model)

### Save it in a txt file 
arch_file   = open("8diamond-LRelu-batchnorm-dropout-kaiming.txt", "w")
file_write  = arch_file.write(model_arch)
arch_file.close()

### log it in your wandb run with wandb.save()
wandb.save('8dpyramids-GELU-batchnorm-dropout.txt')

[]

# Experiment

Now, it is time to finally run your ablations! Have fun!

In [ ]:

def save_ckp(state, is_best, checkpoint_path, best_model_path):
    """
    state: checkpoint we want to save
    is_best: is this the best checkpoint; min validation loss
    checkpoint_path: path to save checkpoint
    best_model_path: path to save best model
    """
    f_path = checkpoint_path
    # save checkpoint data to the path given, checkpoint_path
    torch.save(state, f_path)
    # if it is a best model, min validation loss
    if is_best:
        best_fpath = best_model_path
        # copy that checkpoint file to best path given, best_model_path
        shutil.copyfile(f_path, best_fpath)

In [ ]:
# Iterate over number of epochs to train and evaluate your model
torch.cuda.empty_cache()
gc.collect()
wandb.watch(model, log="all")




for epoch in range(config['epochs']):

    print("\nEpoch {}/{}".format(epoch+1, config['epochs']))

    curr_lr                 = float(optimizer.param_groups[0]['lr'])
    train_loss, train_acc   = train(model, train_loader, optimizer, criterion)
    scheduler.step()
    val_loss, val_acc       = eval(model, val_loader)

    # scheduler.step(val_loss) # for reduce plateu

    print("\tTrain Acc {:.04f}%\tTrain Loss {:.04f}\t Learning Rate {:.07f}".format(train_acc*100, train_loss, curr_lr))
    print("\tVal Acc {:.04f}%\tVal Loss {:.04f}".format(val_acc*100, val_loss))

    ### Log metrics at each epoch in your run 
    # Optionally, you can log at each batch inside train/eval functions 
    # (explore wandb documentation/wandb recitation)
    wandb.log({'train_acc': train_acc*100, 'train_loss': train_loss, 
               'val_acc': val_acc*100, 'valid_loss': val_loss, 'lr': curr_lr})

    ### Highly Recommended: Save checkpoint in drive and/or wandb if accuracy is better than your current best
    torch.save({'epoch': epoch,
              'model_state_dict': model.state_dict(),
              'optimizer_state_dict': optimizer.state_dict(),
              'scheduler_state_dict' : scheduler.state_dict(),
              'loss': val_loss}, 
        '/content/model.pt')
    # torch.save(model.state_dict(), '/content/model.pt')

### Finish your wandb run
run.finish()


Epoch 1/10


Train:   0%|          | 0/15925 [00:00<?, ?it/s]

Val:   0%|          | 0/236 [00:00<?, ?it/s]

	Train Acc 80.5680%	Train Loss 0.5846	 Learning Rate 0.0010000
	Val Acc 83.6381%	Val Loss 0.4792

Epoch 2/10


Train:   0%|          | 0/15925 [00:00<?, ?it/s]

...

	Train Acc 85.1732%	Train Loss 0.4301	 Learning Rate 0.0009945
	Val Acc 85.0488%	Val Loss 0.4353

Epoch 3/10


Train:   0%|          | 0/15925 [00:00<?, ?it/s]

Val:   0%|          | 0/236 [00:00<?, ?it/s]

	Train Acc 86.4411%	Train Loss 0.3873	 Learning Rate 0.0009780
	Val Acc 85.6087%	Val Loss 0.4197

Epoch 4/10


Train:   0%|          | 0/15925 [00:00<?, ?it/s]

In [ ]:
#tips: ReduceLROnPlateau scheduler with Gelu, deeper network-consider stacking pyramids, 

# Testing and submission to Kaggle

Before we get to the following code, make sure to see the format of submission given in *sample_submission.csv*. Once you have done so, it is time to fill the following function to complete your inference on test data. Refer the eval function from previous cells to get an idea of how to go about completing this function.

In [ ]:
def test(model, test_loader):
    ### What you call for model to perform inference?
    model.eval() # TODO train or eval?

    ### List to store predicted phonemes of test data
    test_predictions = []

    ### Which mode do you need to avoid gradients?
    # with torch.inference_mode(): # TODO
    with torch.no_grad():

        for i, mfccs in enumerate(tqdm(test_loader)):
            # print(i, mfccs)

            mfccs   = mfccs.to(device)             
            
            logits  = model(mfccs)

            ### Get most likely predicted phoneme with argmax
            
            predicted_phonemes = torch.argmax(logits, dim= 1)
            predicted_phonemes_array = predicted_phonemes.cpu().detach().numpy()
            # print(predicted_phonemes_array)

            ### How do you store predicted_phonemes with test_predictions? Hint, look at eval 
            test_predictions = test_predictions + predicted_phonemes_array.tolist()

    return test_predictions

In [ ]:
wandb.finish()

In [ ]:
wandb.init(mode="disabled")

In [ ]:
predictions = test(model, test_loader)

In [ ]:
prediction_labels = [Inverse_PHONEMES_MAP[i] for i in predictions]

In [ ]:
### Create CSV file with predictions
with open("/content/data/11-785-s23-hw1p2/submission.csv", "w+") as f:
    f.write("id,label\n")
    for i in range(len(prediction_labels)):
        f.write("{},{}\n".format(i, prediction_labels[i]))

In [ ]:
## Submit to kaggle competition using kaggle API (Uncomment below to use)
!kaggle competitions submit -c 11-785-s23-hw1p2 -f /content/data/11-785-s23-hw1p2/submission.csv -m "Test Submission"

### However, its always safer to download the csv file and then upload to kaggle